In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os

## Inderes recommendations

In [11]:
recommendation = {}
for filename in os.listdir('suositukset'):
    if filename.endswith(".xlsx"): 
        stock = os.path.splitext(filename)[0]
        print(stock)
        filepath = os.path.join('suositukset', filename)
        df_stock = pd.read_excel(filepath) #, parse_dates=[0]
        # Fix the last column where analyst name is in the header
        if len(df_stock.columns) > 4:
            analyst = df_stock.columns[4]
            df_stock.columns = list(df_stock.columns[0:4]) + ['Analyytikko']
            df_stock['Analyytikko'] = analyst
        else:
            df_stock['Analyytikko'] = np.nan
        df_stock['Osake'] = stock
        recommendation[stock] = df_stock

Aallon Group
Admicom
Ahlstrom-Munksjö
Aktia Pankki A
Alexandria Pankkiiriliike
Alma Media
Apetit
Aspo
Aspocomp Group
Avidly
Basware
Bilot
Bittium
Boreo
CapMan
Cargotec
Caverion
Consti
Detection Technology
Digia
Duell
EAB Group
EcoUp
Eezy
Efecte
Elisa
Endomines
Enedo
Enersense
eQ
Etteplan
Evli Pankki
Exel Composites
F-Secure
Fellow Finance
Fifax
Finnair
Fiskars A
Fodelia
Fondia
Fortum
Glaston
Gofore
Harvia
Heeros
Honkarakenne B
Huhtamäki
Ilkka-Yhtymä 2
Incap
Innofactor
Investors House
Kamux
Kemira
Kesko B
Kesla A
Kone
Konecranes
LeadDesk
Lehto Group
Lemonsoft
Loihde
Loudspring
Marimekko
Martela A
Metso Outotec
Metsä Board B
Musti Group
Neste
Netum
Next Games
Nightingale Health
Nixu
NoHo Partners
Nokia
Nokian Renkaat
Norrhydro Group
Nurminen Logistics
Oma Säästöpankki
Optomed
Oriola B
Orion B
Orthex
Outokumpu
Ovaro Kiinteistösijoitus
Panostaja
Partnera
Pihlajalinna
Piippo
Ponsse
PunaMusta Media
Purmo Group
Puuilo
Qt Group
Raisio V
Rapala VMC
Raute A
Reka Industrial
Relais Group
Remedy En

In [13]:
# Translate english to finnish in 'Suominen' stock
df_stock = recommendation["Suominen"]
eng_to_finnish = {"Accumulate":"Lisää", "Reduce":"Vähennä", "Buy":"Osta", "Sell":"Myy", 
                  "Lisää":"Lisää", "Vähennä":"Vähennä", "Osta":"Osta", "Myy":"Myy"}
df_stock["Recommendation"] = df_stock["Recommendation"].map(eng_to_finnish)
df_stock.columns = ['Pvm', 'Suositus', 'Tavoite', 'Osakekurssi', 'Analyytikko', 'Osake']
recommendation["Suominen"] = df_stock

In [17]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()
# Convert SEK to EUR at recommendation date for these stocks
for stock in 'Telia Company', 'Endomines', 'Sotkamo Silver':
    print(stock)
    df_stock = recommendation[stock]
    times = pd.to_datetime(df_stock['Pvm']).dt.to_pydatetime()
    rates = [c.get_rate('SEK', 'EUR', time) for time in times]
    df_stock['Tavoite'] = df_stock['Tavoite'] * rates
    df_stock['Osakekurssi'] = df_stock['Osakekurssi'] * rates
    recommendation[stock] = df_stock

Telia Company
Endomines
Sotkamo Silver


In [24]:
from pandas.api.types import is_datetime64_ns_dtype
for stock in recommendation:
    if not is_datetime64_ns_dtype(recommendation[stock]['Pvm']):
        print(stock)
# Manually check these, 29.2.2019, 29.2.2019, 22.72021, 27.4..2018, 29.4..2021, 2109-02-11 appear wrong

Eezy
Finnair
SRV Yhtiöt
Suominen


In [96]:
recommendations = pd.concat(recommendation.values(), axis=0)

In [97]:
# Fix incorrect dates
date_not = ["29.2.2019", "29,4.2020", "22.72021", "27.4..2018", "29.4..2021"]
date_yes = ["28.2.2019", "29.4.2020", "22.7.2021", "27.4.2018", "29.4.2021"]
recommendations.loc[recommendations['Pvm'].isin(date_not),"Pvm"] = date_yes
# Many dates also mistyped ...
recommendations['Pvm'] = pd.to_datetime(recommendations['Pvm'])
recommendations.loc[recommendations['Pvm'] == pd.Timestamp("2109-02-11"), "Pvm"] = "2019-02-11"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2020-01-12")) & (recommendations['Osake'] == 'Eezy'), "Pvm"] = "2021-01-12"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2021-12-28")) & (recommendations['Osake'] == 'Metsä Board B'), "Pvm"] = "2020-12-28"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2021-01-13")) & (recommendations['Osake'] == 'Apetit'), "Pvm"] = "2022-01-13"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2017-01-12")) & (recommendations['Osake'] == 'Elisa'), "Pvm"] = "2018-01-12"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2018-02-11")) & (recommendations['Osake'] == 'Kemira'), "Pvm"] = "2019-02-11"
recommendations.loc[recommendations['Pvm'].isin([pd.Timestamp("2018-02-06"), pd.Timestamp("2018-04-25")]) & (recommendations['Tavoite'] != 44.0) & (recommendations['Osake'] == 'Kesko B'), "Pvm"] = ["2019-02-06", "2019-04-25"]
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2021-01-13")) & (recommendations['Osake'] == 'Nokian Renkaat'), "Pvm"] = "2022-01-13"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2019-01-21")) & (recommendations['Osake'] == 'Outokumpu'), "Pvm"] = "2020-01-21"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2017-01-15")) & (recommendations['Osake'] == 'Qt Group'), "Pvm"] = "2018-01-15"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2017-02-19")) & (recommendations['Osake'] == 'Remedy Entertainment'), "Pvm"] = "2018-02-19"
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2015-02-26")) & (recommendations['Osake'] == 'Siili Solutions'), "Pvm"] = "2016-02-26"
recommendations.loc[(recommendations['Pvm'].isin([pd.Timestamp("2018-01-17"), pd.Timestamp("2020-11-28")])) & (recommendations['Osake'] == 'Sotkamo Silver'), "Pvm"] = ["2018-07-17", "2020-01-28"]
recommendations.loc[(recommendations['Pvm'] == pd.Timestamp("2018-02-01")) & (recommendations['Osake'] == 'Suominen'), "Pvm"] = "2019-02-01"


In [98]:
recommendations.head()

,Pvm,Suositus,Tavoite,Osakekurssi,Analyytikko,Osake
0,2019-04-09 00:00:00,Vähennä,8.5,8.87,Atte Riikola,Aallon Group
1,2019-08-23 00:00:00,Vähennä,9.0,9.50,Atte Riikola,Aallon Group
2,2019-09-30 00:00:00,Vähennä,9.0,9.34,Atte Riikola,Aallon Group
3,2019-12-12 00:00:00,Vähennä,9.9,10.45,Atte Riikola,Aallon Group
4,2020-02-14 00:00:00,Lisää,11.0,10.45,Atte Riikola,Aallon Group


In [99]:
recommendations.to_csv("Suositukset.csv", index=False, header=True)